 EDA on Amazon Fine Food Review dataset
 ===

# Mount Google Drive

In [129]:
# Mouting Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import Required Modules

In [169]:
import sqlite3
import pandas as pd
import numpy as np
import csv # for CSV file handling
#from tqdm import tqdm_notebook
from tqdm import tqdm
import re # for regular expression over sentences for pre-processing
from nltk.corpus import stopwords # for stopwords removal

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Load Data

In [131]:
# Using sqlite read data from the database
con = sqlite3.connect('/content/drive/My Drive/Colab Notebooks/AFF-Review/database.sqlite')

# Get reviews which do not have score as 3
filtered_data = pd.read_sql_query(""" SELECT * FROM Reviews WHERE Score != 3 """, con)
filtered_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


# Highlevel Statistics

In [132]:
filtered_data.describe()

,Id,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time
count,525814.000000,525814.000000,525814.000000,525814.000000,5.258140e+05
mean,284599.060038,1.747293,2.209544,4.279148,1.295943e+09
std,163984.038077,7.575819,8.195329,1.316725,4.828129e+07
min,1.000000,0.000000,0.000000,1.000000,9.393408e+08
25%,142730.250000,0.000000,0.000000,4.000000,1.270598e+09
50%,284989.500000,0.000000,1.000000,5.000000,1.310861e+09
75%,426446.750000,2.000000,2.000000,5.000000,1.332634e+09
max,568454.000000,866.000000,878.000000,5.000000,1.351210e+09


## Features/ Labels

In [133]:
filtered_data.columns

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')

In [134]:
filtered_data.dtypes

Id                         int64
ProductId                 object
UserId                    object
ProfileName               object
HelpfulnessNumerator       int64
HelpfulnessDenominator     int64
Score                      int64
Time                       int64
Summary                   object
Text                      object
dtype: object

### Observation
- Totally 10 features given
- No labels given
- From Kaggle below information I have obtained about teach feature
  - https://www.kaggle.com/snap/amazon-fine-food-reviews
- Id
  - Row Id
- ProductId
  - Unique identifier for the product
- UserId
  - Unqiue identifier for the user
- ProfileName
  - Profile name of the user
- HelpfulnessNumerator
  - Number of users who found the review helpful
- HelpfulnessDenominator
  - Number of users who indicated whether they found the review helpful
- Score
  - Rating between 1 and 5
- Time
  - Timestamp for the review
- Summary
  - Brief summary of the review
- Text
  - Text of the review

# Data Cleaning

## Analysis

### Id

In [135]:
u = filtered_data.Id.value_counts()
u.unique()

array([1])

#### Observation
- No Id repeation

### ProductId

In [136]:
len(filtered_data.ProductId.unique())

72005

#### Observation
- 72005 Products

### UserId

In [137]:
len(filtered_data.UserId.unique())

243414

#### Observation
- 243414 Users

### HelpfulnessNumerator 

In [138]:
print(filtered_data.HelpfulnessNumerator.min(),
      filtered_data.HelpfulnessNumerator.max(),
      len(filtered_data.HelpfulnessNumerator.unique()))

0 866 222


#### Observation
- value ranges from 0 to 808
- 222 unique entries

### HelpfulnessDenominator

In [139]:
print(filtered_data.HelpfulnessDenominator.min(),
      filtered_data.HelpfulnessDenominator.max(),
      len(filtered_data.HelpfulnessDenominator.unique()))

0 878 227


In [140]:
# As per feature details, Denominator should be greater than Numerator
# Lets check whether the data follows that description
filtered_data[(filtered_data.HelpfulnessDenominator < filtered_data.HelpfulnessNumerator)]

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
41159,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...
59301,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...


#### Observation
- value ranges from 0 to 878
- 227 unique entries
- **2 invalid entries found**
  - Denominator is greater than Numerator

### Score

In [141]:
filtered_data.Score.unique()

array([5, 1, 4, 2])

In [142]:
filtered_data.Score.value_counts()

5    363122
4     80655
1     52268
2     29769
Name: Score, dtype: int64

#### Observation
- Scores range from 1 to 5 only
- No invalid entries found
- **No equal amount of data points for each score**
  - We have an IMBALANCED dataset

### Time

In [143]:
len(filtered_data.Time.unique())

3157

In [0]:
#filtered_data['Time'].value_counts()

In [0]:
# Check whether any entry with same time for more than one product
# which is practically not possible
userid_group = filtered_data.groupby('UserId')
#g = userid_group.groups
#g.values()

In [146]:
userid_group.filter(lambda x:len(x)>1).sort_values('Time')

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
346055,374359,B00004CI84,A344SMIA5JECGM,Vincent P. Ross,1,2,5,944438400,A modern day fairy tale,"A twist of rumplestiskin captured on film, sta..."
417859,451878,B00004CXX9,A344SMIA5JECGM,Vincent P. Ross,1,2,5,944438400,A modern day fairy tale,"A twist of rumplestiskin captured on film, sta..."
212472,230285,B00004RYGX,A344SMIA5JECGM,Vincent P. Ross,1,2,5,944438400,A modern day fairy tale,"A twist of rumplestiskin captured on film, sta..."
346116,374422,B00004CI84,A1048CYU0OV4O8,Judy L. Eans,2,2,5,947376000,GREAT,THIS IS ONE MOVIE THAT SHOULD BE IN YOUR MOVIE...
417927,451949,B00004CXX9,A1048CYU0OV4O8,Judy L. Eans,2,2,5,947376000,GREAT,THIS IS ONE MOVIE THAT SHOULD BE IN YOUR MOVIE...
212533,230348,B00004RYGX,A1048CYU0OV4O8,Judy L. Eans,2,2,5,947376000,GREAT,THIS IS ONE MOVIE THAT SHOULD BE IN YOUR MOVIE...
417847,451864,B00004CXX9,A1B2IZU1JLZA6,Wes,19,23,1,948240000,WARNING: CLAMSHELL EDITION IS EDITED TV VERSION,"I, myself always enjoyed this movie, it's very..."
212458,230269,B00004RYGX,A1B2IZU1JLZA6,Wes,19,23,1,948240000,WARNING: CLAMSHELL EDITION IS EDITED TV VERSION,"I, myself always enjoyed this movie, it's very..."
346041,374343,B00004CI84,A1B2IZU1JLZA6,Wes,19,23,1,948240000,WARNING: CLAMSHELL EDITION IS EDITED TV VERSION,"I, myself always enjoyed this movie, it's very..."
346141,374450,B00004CI84,ACJR7EQF9S6FP,Jeremy Robertson,2,3,4,951523200,Bettlejuice...Bettlejuice...BETTLEJUICE!,What happens when you say his name three times...


In [0]:
#filtered_data[filtered_data['Summary'].str.contains('book')]
#type(filtered_data[filtered_data['Summary'].str.contains('book')].index.tolist())

#suspicious_indices = []
#
#l = filtered_data[filtered_data['Summary'].str.contains('book')].index.tolist()
#print("No. of entries having '{0}' is {1}".format('book', len(l)))
#suspicious_indices = suspicious_indices + l
#
#l = filtered_data[filtered_data['Summary'].str.contains('film')].index.tolist()
#print("No. of entries having '{0}' is {1}".format('film', len(l)))
#suspicious_indices = suspicious_indices + l
#
#l = filtered_data[filtered_data['Summary'].str.contains('Film')].index.tolist()
#print("No. of entries having '{0}' is {1}".format('Film', len(l)))
#suspicious_indices = suspicious_indices + l
#
#l = filtered_data[filtered_data['Summary'].str.contains('Book')].index.tolist()
#print("No. of entries having '{0}' is {1}".format('Book', len(l)))
#suspicious_indices = suspicious_indices + l

def getEntriesHavingTexts(df, col_to_search, text_list):
  indices = []
  counts = []
  for text in text_list:
    l = filtered_data[filtered_data[col_to_search].str.contains(text)].index.tolist()
    counts.append(len(l))
    indices = indices + l
  return indices, counts

In [148]:
text_list = ['[bB]ook']
suspicious_indices, counts = getEntriesHavingTexts(filtered_data,
                                       'Summary',
                                       text_list)

for i in range(len(counts)):
  print("No. of entries having '{0}' is {1}".format(text_list[i], counts[i]))

print('Total suspicious entries : ', len(suspicious_indices))
save_data = filtered_data.iloc[suspicious_indices]
save_data.to_csv('test_1.csv')

No. of entries having '[bB]ook' is 85
Total suspicious entries :  85


In [149]:
text_list = ['[fF]ilm']
suspicious_indices, counts = getEntriesHavingTexts(filtered_data,
                                       'Summary',
                                       text_list)

for i in range(len(counts)):
  print("No. of entries having '{0}' is {1}".format(text_list[i], counts[i]))


print('Total suspicious entries : ', len(suspicious_indices))
save_data = filtered_data.iloc[suspicious_indices]
save_data.to_csv('test_2.csv')

No. of entries having '[fF]ilm' is 24
Total suspicious entries :  24


In [150]:
# Found 'Tim Burton' movies reviews in Food Reviews
text_list = ['Tim Burton']
suspicious_indices, counts = getEntriesHavingTexts(filtered_data,
                                       'Summary',
                                       text_list)

for i in range(len(counts)):
  print("No. of entries having '{0}' is {1}".format(text_list[i], counts[i]))


print('Total suspicious entries : ', len(suspicious_indices))
save_data = filtered_data.iloc[suspicious_indices]
save_data.to_csv('Tim_Burton_2.csv')

No. of entries having 'Tim Burton' is 36
Total suspicious entries :  36


#### Observation
- There are duplicates
  - Same user having review comments for more than one product at same timestamp which is impractical

## Cleaning

### Convert Score to Numerical Value 0/1 for negative/positive review

In [151]:
def ScoreToReviewType(score):
  if score < 3:
    return 0
  return 1

filtered_data.Score = filtered_data.Score.map(ScoreToReviewType)
print(filtered_data.Score.unique())

[1 0]


### Drop Duplicates

In [0]:
# Sort the data based on ProductID in ascending order so that we can keep only one kind of product review
sorted_data = filtered_data.sort_values('ProductId',axis=0, ascending=True, inplace=False, na_position='last')

In [153]:
# keep first entry, drop remaining duplicate entries
final_data = sorted_data.drop_duplicates(subset={'UserId','ProfileName','Time','Text'},keep='first',inplace=False)
print(final_data.shape)

(364173, 10)


### Remove invalid Helpfull Score entries

In [154]:
final_data = final_data[final_data.HelpfulnessNumerator <= final_data.HelpfulnessDenominator]
print(final_data.shape)

(364171, 10)


### Remove Invalid Summary Entries

#### Remove actual film reviews
- Tim Burton (found by filtering film words and looking into data)

In [155]:
final_data = final_data[~final_data.Summary.str.contains('Tim Burton')]
print(final_data.shape)

(364159, 10)


In [156]:
final_data = final_data[~final_data.Text.str.contains('Tim Burton')]
print(final_data.shape)

(364106, 10)


### Analyse for any invalid entries in review text

```
def getUniqueWords(df, col_name):
    words = set()
    #words.add(' ')
    count  = 0
    for index, row in tqdm(df.iterrows()):
        w_l = list(set(row[col_name].split()))
        words = words.union(set(w_l))
        #print(row[col_name], w_l)
        #print(list(words))
        count += 1
        #if count > 20:
        #    break
    return words
'''

"""
tt = final_data[~final_data.Summary.str.isalpha()]
print(tt.shape)
tt.apply()
"""

%%time
summary_words = getUniqueWords(final_data, 'Summary')

tqdm(text_words = getUniqueWords(final_data, 'Text'))

print('Total unique words in Summary: ', len(summary_words))
print('Total unique words in Review Text: ', len(text_words))

def storeSet_1(w_set, file_name):
    #csv_file = csv.writer(open(file_name), 'w')
    with open(file_name, 'w', encoding="utf-8") as csv_file:
        cw = csv.writer(csv_file)
        cw.writerow(list(w_set))
        
def storeSet_2(w_set, file_name):
    with open(file_name, 'w', encoding="utf-8") as csv_file:
        for w in w_set:
            csv_file.write(w)
            csv_file.write('\n')

storeSet_2(summary_words, 'summary_words.csv')
storeSet_2(text_words, 'text_words.csv')

import string 

invalidChars = set(string.punctuation.replace("_", ""))

def containsAny(word, char_list):
    '''
    If any of the character in char_list found in 'word' will return True
    Otherwise returns False
    '''
    for c in char_list:
        if c in word:
            return True
    return False

def containsAll(word, char_list):
    '''
    If all of the characters in char_list found in 'word' will return True
    Otherwise returns False
    '''
    for c in char_list:
        if c not in word:
            return True
    return False

def getWordsHavingSpecialChar(df, col_name):
    words = set()
    #words.add(' ')
    count  = 0
    for index, row in df.iterrows():
        w_l = list(set(row[col_name].split()))
        w_c_l = []
        for w in w_l:
            if containsAny(w, invalidChars):
                w_c_l.append(w)
        words = words.union(set(w_c_l))
        #print(row[col_name], w_l)
        #print(list(words))
        #count += 1
        #if count > 20:
        #    break
    return words

%%time
summary_invalid_words = getWordsHavingSpecialChar(final_data, 'Summary')

%%time
text_invalid_words = getWordsHavingSpecialChar(final_data, 'Text')

print('Total unique (invalid) words in Summary: ', len(summary_invalid_words))
print('Total unique (invalid) words in Review Text: ', len(text_invalid_words))

storeSet_2(summary_invalid_words, 'summary_invalid_words.csv')
storeSet_2(text_invalid_words, 'text_invalid_words.csv')

## Remove Invalid Entries

In [157]:
def removeHtmlTags(sentence):
  '''
  function to remove HTML tags in the given sentence
  '''
  reg_exp = re.compile('<.*?>', )
  cleaned_text = re.sub(reg_exp, ' ', sentence)
  return cleaned_text

def removePunctuations(sentence):
  '''
  function to remove punctuations in the given sentence
  '''
  cleaned_sentence = re.sub(r'[?|!|\'|"|#]',r'',sentence)
  cleaned_sentence = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned_sentence)
  return cleaned_sentence

'''
s = 'Hi I am <pr> test </pr> testing'
removeHtmlTags(s).split()
'''

"\ns = 'Hi I am <pr> test </pr> testing'\nremoveHtmlTags(s).split()\n"

In [0]:
stop_words = set(stopwords.words('english')) # get stop words for English
#print(stop)
snow_stem = nltk.stem.SnowballStemmer('english') # get Stemmer for English
#print(snow)

In [170]:
all_positive_words = []
all_negative_words = []
final_review_texts = []
df_index = 0 # for tracking the observations

for sent in tqdm(final_data['Text'].values):
  #print('{0} ==> '.format(df_index), sent)
  sent = removeHtmlTags(sent) # remove HTML tags first
  #print('{0} ==> '.format(df_index), sent)
  
  filtered_words = []
  for w in sent.split():
    #print(removePunctuations(w))
    for cleaned_word in removePunctuations(w).split():
      if ((cleaned_word.isalpha()) & (len(cleaned_word) > 2)):
        cleaned_word = cleaned_word.lower()
        #print(cleaned_word)
        if (cleaned_word not in stop_words):
          s = (snow_stem.stem(cleaned_word)).encode('utf8')
          filtered_words.append(s)
          if (final_data['Score'].values)[df_index] == 1:
            all_positive_words.append(s)
          else:
            all_negative_words.append(s)
        else:
          continue
      else:
        continue
  filtered_sent = b" ".join(filtered_words)
  #print(filtered_words, filtered_sent)
  
  final_review_texts.append(filtered_sent)
  
  #df_index += 1
  #if df_index > 10:
  #  break






  0%|          | 0/364106 [00:00<?, ?it/s]



  0%|          | 84/364106 [00:00<07:14, 838.74it/s]



  0%|          | 181/364106 [00:00<06:57, 871.82it/s]



  0%|          | 267/364106 [00:00<07:00, 865.83it/s]



  0%|          | 360/364106 [00:00<06:51, 883.36it/s]



  0%|          | 429/364106 [00:00<07:27, 812.04it/s]



  0%|          | 498/364106 [00:00<10:10, 595.20it/s]



  0%|          | 560/364106 [00:00<10:09, 596.70it/s]



  0%|          | 620/364106 [00:00<10:09, 596.32it/s]



  0%|          | 699/364106 [00:00<09:26, 641.68it/s]



  0%|          | 768/364106 [00:01<09:16, 652.87it/s]



  0%|          | 854/364106 [00:01<08:36, 702.69it/s]



  0%|          | 944/364106 [00:01<08:03, 751.36it/s]



  0%|          | 1028/364106 [00:01<07:48, 775.08it/s]



  0%|          | 1108/364106 [00:01<09:03, 667.81it/s]



  0%|          | 1179/364106 [00:01<10:01, 603.14it/s]



  0%|          | 1257/364106 [00:01<09:21, 645.90it/s]



  0%|          | 1340/364106 [00:01

In [171]:
# add cleaned text as a seperate column into our final data dataframe
final_data['CleanedText'] = final_review_texts
final_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText
138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,1,939340800,EVERY book is educational,this witty little book makes my son laugh at l...,b'witti littl book make son laugh loud recit c...
138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,1,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc...",b'grew read sendak book watch realli rosi movi...
138689,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,1,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...,b'fun way children learn month year learn poem...
138690,150508,0006641040,AZGXZ2UUK6X,"Catherine Hallberg ""(Kate)""",1,1,1,1076025600,a good swingy rhythm for reading aloud,This is a great little book to read aloud- it ...,b'great littl book read nice rhythm well good ...
138691,150509,0006641040,A3CMRKGE0P909G,Teresa,3,4,1,1018396800,A great way to learn the months,This is a book of poetry about the months of t...,b'book poetri month year goe month cute littl ...


In [0]:
# store final data into new database
conn = sqlite3.connect('cleaned.sqlite')
c = conn.cursor()
conn.text_factory = str
final_data.to_sql('Reviews', conn, schema=None, if_exists='replace', 
                  index=True, index_label=None, dtype=None)
conn.close()

In [176]:
!pwd

/content


In [0]:
!mv "/content/cleaned.sqlite" "/content/drive/My Drive/Colab Notebooks/AFF-Review/cleaned.sqlite"

In [180]:
!ls

drive  sample_data  test_1.csv	test_2.csv  Tim_Burton_2.csv


## Observation Summary

- TO DO